In [171]:
import requests as req
from bs4 import BeautifulSoup
import pandas as pd
import os

In [ ]:
############################################################
#    !IMPORTANT!   CONFIGURE SCRIPT IN THIS CELL
############################################################

#Local target for data files
path = '/home/maile/Desktop/doughfiles/'

#Number of pinterest recipes to load
target = 500

# Pinterest login info
pinterest_ux = 'mailecupo@gmail.com'
pinterest_pw = 'Morgantheparrot!@!'

In [186]:
found = true
while found:
home = req.get('https://www.foodnetwork.com/search/pizza-dough-/p/{n}'.format(n = n)).text
soup = BeautifulSoup(home, 'html.parser')
for a in soup.find_all('a'):
    for child in a.find_all():
        if child.get('class')=='m-MediaBlock__a-HeadlineText':
            l.append(a)
            break

In [204]:
############################################################
#    Get recipelist from foodnetwork
############################################################

l = []
n=0

while True:
    n+=1
    print('https://www.foodnetwork.com/search/pizza-dough-/p/{n}'.format(n = n))
    if home.find('Sorry, no results')>-1:
        break #Exit while loop
    soup = BeautifulSoup(home, 'html.parser')
    for a in soup.find_all('a'):
        for child in a.find_all():
            if child.get('class'):
                if child.get('class')[0] == 'm-MediaBlock__a-HeadlineText':
                    l.append(a.get('href'))
                    print('https:'+a.get('href')) #for some reason the protocol is left out of the url
                    break

https://www.foodnetwork.com/search/pizza-dough-/p/1
//www.foodnetwork.com/recipes/bobby-flay/pizza-dough-recipe-1921714
//www.foodnetwork.com/recipes/tyler-florence/pizza-dough-recipe-1914764
//www.foodnetwork.com/recipes/food-network-kitchen/basic-pizza-dough-recipe-1973255
//www.foodnetwork.com/recipes/food-network-kitchen/two-ingredient-pizza-dough-3542644
//www.foodnetwork.com/kitchen/classes/2-ingredient-pizza-dough_64102a19-60b6-45bf-82d2-f4e02522659e
//www.foodnetwork.com/kitchen/classes/perfect-every-time-pizza-dough-and-pepperoni-pizza-_c560c6fa-65a1-4f2d-bf0b-acdf983c3e91
//www.foodnetwork.com/kitchen/classes/white-flour-master-dough-for-focaccia-and-pizza_0825dd21-d312-432a-ba02-2f46876b039a
//www.foodnetwork.com/recipes/giada-de-laurentiis/pizza-dough-3075408
//www.foodnetwork.com/recipes/food-network-kitchen/2-ingredient-grilled-pizza-dough-8806620
//www.foodnetwork.com/recipes/ree-drummond/foolproof-pizza-dough-5450502
//www.foodnetwork.com/recipes/jamie-oliver/pizza-doug

In [212]:
############################################################
#    Get ingredientlists from foodnetwork recipes
############################################################

df = pd.DataFrame()

for r in l:
    home = req.get('https:'+r).text
    soup = BeautifulSoup(home, 'html.parser')
    for i in soup.findAll("p",{"class": "o-Ingredients__a-Ingredient"}):
        df = df.append({'recipe':'https:'+r, 'ingredient':i.text.strip(), 'source':'foodnetwork'}, ignore_index = True)
        
df.to_csv(os.path.join(path,'foodnetwork.csv'))

In [173]:
############################################################
#    Get List of Recipes from allrecipes
############################################################

l=[]
n = 0
found = True

while found:
    found = False
    n+=1
    home = req.get('https://www.allrecipes.com/recipes/1035/bread/pizza-dough-and-crusts/?page={n}'.format(n = n)).text
    soup = BeautifulSoup(home, 'html.parser')
    for a in soup.find_all('a'):
        if a.get('class'):
            if a.get('class')[0] == 'fixed-recipe-card__title-link':
                found = True
                l.append(a.get('href'))                

In [174]:
############################################################
#    Get ingredientlist from allrecipes
############################################################
df = pd.DataFrame()

for r in l:
    home = req.get(r).text
    soup = BeautifulSoup(home, 'html.parser')
    for e in soup.find_all(class_ = 'ingredients-item-name'):
        df = df.append({'recipe':r, 'ingredient':e.text.strip(), 'source':'allrecipes'}, ignore_index = True)

df.to_csv(os.path.join(path,'allrecipes.csv'))

In [131]:
############################################################
#    Get recipes from pinterest
############################################################

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep

driver = webdriver.Firefox()

# Login to pinterest
driver.get("https://www.pinterest.com/login/")
sleep(3)
soup = BeautifulSoup(driver.page_source, 'html.parser')
if driver.find_element_by_id('email'):
    driver.find_element_by_id('email').send_keys(pinterest_ux)
if driver.find_element_by_id('password'):
    driver.find_element_by_id('password').send_keys(pinterest_pw)
sleep(6)
driver.find_elements_by_xpath('//*[@data-test-id="registerFormSubmitButton"]')[0].click()
sleep(6)

#Get pizza dough pin page
driver.get("https://www.pinterest.com/search/pins/?q=pizza%20dough")
sleep(3)

# Scroll down until threshold is reached
h = 0
s=set()

while len(s) < target:
    h+= 500*random.random()+4000 #Scrolls between 1000 and 2000 pixels
    driver.execute_script("window.scrollTo(0, {h})".format(h = h))
    print('Scrolling to '+str(h))
    sleep(4*random.random()+3) # Waits between 3 and 7 seconds
    
    # Get pins on page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    for a in soup.find_all('a'):
        if a.get('href'):
            if a.get('href').find('/pin')>-1:
                s.add(a.get('href'))
    
    print(str(len(s)) + ' pins found.')

driver.close()

Scrolling to 4068.6779363158016
31
Scrolling to 8328.852171120152
53
Scrolling to 12504.891755658533
93
Scrolling to 16769.15442273653
132
Scrolling to 20813.51981064914
167
Scrolling to 25272.559666801266
186
Scrolling to 29434.652930457036
218
Scrolling to 33824.407195243264
249
Scrolling to 38262.648569534154
292
Scrolling to 42316.80423581714
319
Scrolling to 46505.53907437579
362
Scrolling to 50780.71753443519
385
Scrolling to 55266.83024602682
397
Scrolling to 59378.865862705214
410
Scrolling to 63692.91505778558
431
Scrolling to 67896.20646322858
442
Scrolling to 72198.42233797582
468
Scrolling to 76494.07291805065
484
Scrolling to 80811.03490990392
529


In [154]:
############################################################
#    Get ingredients from pinterest
############################################################

driver = webdriver.Firefox()

driver.get("https://www.pinterest.com/login/")
sleep(3)
soup = BeautifulSoup(driver.page_source, 'html.parser')
if driver.find_element_by_id('email'):
    driver.find_element_by_id('email').send_keys(pinterest_ux)
if driver.find_element_by_id('password'):
    driver.find_element_by_id('password').send_keys(pinterest_pw)
sleep(6)
driver.find_elements_by_xpath('//*[@data-test-id="registerFormSubmitButton"]')[0].click()
sleep(6)

for pin in s:
    sleep(5)
    page = 'http://www.pinterest.com{0}'.format(pin)
    driver.get(page)
    print(page)
    
    for ingredient in driver.find_elements_by_xpath('//*[@itemprop="recipeIngredient"]'):
        df = df.append({'recipe':page, 'ingredient':ingredient.text.strip(), 'source':'pinterest'}, ignore_index = True)
        
driver.close()

df.to_csv(os.path.join(path, 'pinterest.csv'))

http://www.pinterest.com/pin/864268984720160622/
http://www.pinterest.com/pin/400046379403764210/
http://www.pinterest.com/pin/215680269644187320/
3/4 tsp Garlic powder
1/2 tsp Onion powder
1 cup All-purpose flour, unbleached
1 1/2 tsp Baking powder
1/2 tsp Kosher salt
1 cup Greek yogurt, nonfat plain
http://www.pinterest.com/pin/46795283616432550/
2 1/4 tsp Active dry yeast
2 1/2 cups All-purpose flour
1 tsp Granulated sugar
1 tsp Kosher salt
1/4 tsp Lawry's garlic salt
2 tbsp Olive oil, extra virgin
http://www.pinterest.com/pin/436919601352213631/
http://www.pinterest.com/pin/511651207669269330/
http://www.pinterest.com/pin/507288345528800817/
6 cups Flour
2 1/2 tsp Salt
2 tsp Sugar
1 tsp Yeast, instant dry
1 tbsp Olive oil
2 1/4 cups Water
http://www.pinterest.com/pin/237705686569138667/
http://www.pinterest.com/pin/157696424440423360/
1 tbsp Honey
3/4 tsp Active dry yeast
3 3/4 cup Bread flour
1 1/2 tsp Salt
2 tbsp Olive oil
1 1/2 cup Sourdough starter discard
3/4 cup Water
http://

TimeoutException: Message: Timeout loading page after 300000ms
